In [ ]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import tensorflow as tf
import os
from tqdm import tqdm

source_path = (os.path.abspath(os.path.join(os.path.dirname("__file__"), '..'))+ '/src/')
sys.path.append(source_path)

from predict_all import *

gpu_number = DEVICE_NUMBER #### SET GPU number 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
	tf.config.experimental.set_visible_devices(gpus[gpu_number], 'GPU') 
	logical_gpus = tf.config.experimental.list_logical_devices('GPU')

In [ ]:
main_dir = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_dir = os.path.join(main_dir, "data/")
models_dir = os.path.join(main_dir, "model/")
outputs_dir = models_dir = os.path.join(main_dir, "outputs/")

### Gerekliliklerin Yüklenmesi
- <b>word_index:</b> <i>{'resmi': 1, 'gazete': 2, 'tarihi': 3, 'sayısı': 4, 'kanun': 5, .... }</i>
- <b>label_dict:</b> <i>{'Cumhurbaşkanlığı Kararnamesi': 0, 'Genelge': 1, 'Kanun': 2, 'Kanun Hükmünde Kararname': 3, ...}</i>

In [ ]:
# formed as a result of training
model_name = "model.hdf5"
word_index_name = "word_index.pkl"
label_dict_name = "label_dict.pkl"

In [ ]:
# load data, model, word_index and label_dict
df = pd.read_csv(data_dir + "ornek-eval-dataset-yarismacilara-verilecek.csv")

model = load_model(models_dir + f"{model_name}")
word_index = pickle.load(open(data_dir + f"{word_index_name}", "rb"))
label_dict = pickle.load(open(data_dir + f"{label_dict_name}", "rb"))

### Tahminleme

In [ ]:
#kanun:       rega_no, mukerrer_no, rega_tarihi, mevzuat_tarihi, mevzuat_no, len(matching_madde_list)
#khk:         rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, len(matching_madde_list)
#cumhur:      rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, len(matching_madde_list)
#tuzuk: 	  rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, len(matching_madde_list)
#yonetmelik:  rega_no, mukerrer_no, rega_tarihi, len(matching_madde_list)
#komisyon:    sira_no, donem
#genelge:     mevzuat_tarihi, mevzuat_no, belge_sayi
#ozelge:      mevzuat_tarihi
#tebliğ:      rega_no, mukerrer_no, rega_tarihi
#resmi:       rega_no, mukerrer_no, rega_tarihi

In [ ]:

# dataframe for results
results_df = pd.DataFrame(columns=["data_text", "kategori", "rega_no", "mukerrer_no", "rega_tarihi", "mevzuat_no", "belge_sayi", "mevzuat_tarihi", "donem", "sira_no", "madde_sayisi", "kurum"])
results_df.loc[0, "mukerrer_no"] = 0
results_df.loc[0, "madde_sayisi"] = 0
results_df.loc[0, "kurum"] = np.NaN

for i, text in enumerate(tqdm(df.data_text.values)):
    results_df.loc[i, "data_text"] = text
    kategori_prediction = predict_category(model, word_index, label_dict, 64, text)
    results_df.loc[i, "kategori"] = kategori_prediction
    
    if kategori_prediction == "Kanun Hükmünde Kararname":
        rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi = extract_kanun_hukmunde_kararname_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        results_df.loc[i, "mevzuat_no"] = mevzuat_no
        results_df.loc[i, "madde_sayisi"] = madde_sayisi
    
    if kategori_prediction == "Cumhurbaşkanlığı Kararnamesi":
        rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi = extract_cumhurbaskanligi_kararnamesi_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        results_df.loc[i, "mevzuat_no"] = mevzuat_no
        results_df.loc[i, "madde_sayisi"] = madde_sayisi
    
    if kategori_prediction == "Genelge":
        mevzuat_tarihi, mevzuat_no, belge_sayi = extract_genelge_info(text)
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        results_df.loc[i, "belge_sayi"] = belge_sayi
        results_df.loc[i, "mevzuat_no"] = mevzuat_no
        
    if kategori_prediction == "Kanun":
        rega_no, mukerrer_no, rega_tarihi, mevzuat_tarihi, mevzuat_no, madde_sayisi = extract_kanun_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        results_df.loc[i, "mevzuat_no"] = mevzuat_no
        results_df.loc[i, "madde_sayisi"] = madde_sayisi
        
    if kategori_prediction == "Komisyon Raporu":
        sira_no, donem = extract_komisyon_raporu_info(text)
        results_df.loc[i, "sira_no"] = sira_no
        results_df.loc[i, "donem"] = donem
        
    if kategori_prediction == "Resmi Gazete":
        rega_no, mukerrer_no, rega_tarihi = extract_resmi_gazete_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
    
    if kategori_prediction == "Yönetmelik":
        rega_no, mukerrer_no, rega_tarihi = extract_teblig_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        
    if kategori_prediction == "Tebliğ":
        rega_no, mukerrer_no, rega_tarihi = extract_teblig_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        
    if kategori_prediction == "Tüzük":
        rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi = extract_tuzuk_info(text)
        results_df.loc[i, "rega_no"] = rega_no
        results_df.loc[i, "mukerrer_no"] = mukerrer_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        results_df.loc[i, "mevzuat_no"] = mevzuat_no
        results_df.loc[i, "rega_tarihi"] = rega_tarihi
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        results_df.loc[i, "madde_sayisi"] = madde_sayisi  
        
    if kategori_prediction == "Özelge":
        mevzuat_tarihi = extract_ozelge_info(text)
        results_df.loc[i, "mevzuat_tarihi"] = mevzuat_tarihi
        
results_df.madde_sayisi = results_df.madde_sayisi.fillna(0)
results_df.mukerrer_no = results_df.mukerrer_no.fillna(0)    


In [ ]:
results_df.to_csv(outputs_dir + "ornek-eval-dataset.csv", index=None)